## convolutional layers

convolutional layers are very often used in image recognition tasks for machine learning.
In this notebook the dense layers are replaced by convolutional layers to test this claim.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In this notebook two 2-dimensional convolutional layers are used.
The padding is set to 'same' to keep the output shape the same.
The filter size is set to eight for both layers and the kernel size is four by four.

These hyper-parameters are again set by a little manual testing to provide a somewhat reasonable result.

The number of parameters is smaller than the dense layer (the previous models had 33955 parameter).

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(8, (4,4), activation='relu', padding='same', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(8, (4,4), activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 8)         136       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 8)         1032      
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 24579     
Total params: 25,747
Trainable params: 25,747
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(lr=0.003, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', 'srp64', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
20/20 [==============================] - 3s 127ms/step - loss: 1.1126 - acc: 0.3325
Epoch 2/20
20/20 [==============================] - 2s 78ms/step - loss: 1.0889 - acc: 0.4000
Epoch 3/20
20/20 [==============================] - 2s 80ms/step - loss: 1.0622 - acc: 0.4800
Epoch 4/20
20/20 [==============================] - 1s 67ms/step - loss: 1.0377 - acc: 0.5075
Epoch 5/20
20/20 [==============================] - 1s 68ms/step - loss: 0.9722 - acc: 0.6500
Epoch 6/20
20/20 [==============================] - 1s 65ms/step - loss: 0.9004 - acc: 0.6825
Epoch 7/20
20/20 [==============================] - 2s 76ms/step - loss: 0.8367 - acc: 0.6750
Epoch 8/20
20/20 [==============================] - 1s 70ms/step - loss: 0.6837 - acc: 0.7525
Epoch 9/20
20/20 [==============================] - 1s 74ms/step - loss: 0.6896 - acc: 0.7425
Epoch 10/20
20/20 [==============================] - 1s 72ms/step - loss: 0.6105 - acc: 0.7350
Epoch 11/20
20/20 [==============================] - 1s 74

In [6]:
model.evaluate_generator(test_generator)

[0.44487083454926807, 0.85333335]

The resulting model is smaller than the previous models (295kb to 232kb) and is more accurate with a lower loss. Therefore it seems that convolutional layer are indeed superior then simple dense layer for image recognition.

In [7]:
model_path = join('models', 'devel', 'srp64.h5')
model.save(model_path)